In [1]:
print('Hiii')

Hiii


In [2]:
# ! pip install OCC-core
import sys
print(sys.version)

3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:07:43) [MSC v.1942 64 bit (AMD64)]


In [ ]:
# first create an environment 
# conda create --name=pyoccenv python=3.10
# source activate pyoccenv
# conda install -c conda-forge pythonocc-core=7.8.1.1

In [1]:
from OCC.Core.STEPControl import STEPControl_Reader
from OCC.Core.IFSelect import IFSelect_RetDone
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_EDGE, TopAbs_FACE, TopAbs_SOLID, TopAbs_VERTEX
from OCC.Core.TopoDS import TopoDS_Edge, TopoDS_Face, TopoDS_Solid, TopoDS_Vertex
from OCC.Core.BRep import BRep_Tool
from OCC.Core.Quantity import Quantity_ColorRGBA
from OCC.Core.XCAFDoc import XCAFDoc_ColorType
from OCC.Core.TDF import TDF_LabelSequence
from OCC.Core.TDocStd import TDocStd_Document
from OCC.Core.XCAFApp import XCAFApp_Application
from OCC.Core.BRepGProp import BRepGProp
from OCC.Core.GProp import GProp_GProps
from OCC.Core.BRepTools import BRepTools
import os

# Function to extract all data from STEP file
def extract_all_data(step_file, output_txt):
    # Load the STEP file
    doc = TDocStd_Document("")
    app = XCAFApp_Application.GetApplication()
    app.NewDocument("", doc)
    reader = STEPControl_Reader()
    status = reader.ReadFile(step_file)

    if status != IFSelect_RetDone:
        print("Error: Cannot read STEP file.")
        return

    reader.Transfer(doc.Main())

    # Get shape
    shape = reader.OneShape()

    # Open a text file to store data
    with open(output_txt, "w") as file:
        file.write(f"STEP File: {os.path.basename(step_file)}\n\n")

        # Define shape types to explore
        shape_types = {
            "Edges": (TopAbs_EDGE, "Edge"),
            "Faces": (TopAbs_FACE, "Face"),
            "Solids": (TopAbs_SOLID, "Solid"),
            "Vertices": (TopAbs_VERTEX, "Vertex"),
        }

        for shape_name, (shape_type, label) in shape_types.items():
            file.write(f"--- {shape_name} ---\n")

            explorer = TopExp_Explorer(shape, shape_type)
            count = 0
            while explorer.More():
                item = explorer.Current()

                # Get geometry details
                gprops = GProp_GProps()
                BRepGProp.LinearProperties(item, gprops)
                center = gprops.CentreOfMass()

                # Save shape details
                file.write(f"{label} {count}: Center (X={center.X()}, Y={center.Y()}, Z={center.Z()})\n")

                # Get color (if available)
                color_tool = doc.Main().GetObject().ColorTool()
                color_label_seq = TDF_LabelSequence()
                color_tool.GetColors(XCAFDoc_ColorType.XCAFDoc_ColorGen, color_label_seq)

                color_found = False
                for i in range(color_label_seq.Length()):
                    label = color_label_seq.Value(i + 1)
                    if color_tool.IsSet(label, XCAFDoc_ColorType.XCAFDoc_ColorGen):
                        color = Quantity_ColorRGBA()
                        color_tool.GetColor(label, XCAFDoc_ColorType.XCAFDoc_ColorGen, color)
                        file.write(f"  - Color: (R={color.Red()}, G={color.Green()}, B={color.Blue()})\n")
                        color_found = True
                
                if not color_found:
                    file.write("  - Color: Not Found\n")

                count += 1
                explorer.Next()

            file.write(f"Total {shape_name}: {count}\n\n")

    print(f"All STEP data saved to {output_txt}")

# Run the function with a sample STEP file
step_file_path = "17813_CSI-2_FPC-Cable_120-TF3xFH55_Curved.step"  # Replace with your actual file
output_txt_path = "step_data.txt"

extract_all_data(step_file_path, output_txt_path)

: 